In [ ]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
splitedRdd.take(1)
#(uid,[[train],[test]]

In [12]:
#import time
#import datetime
#from time import mktime  
from datetime import datetime
from operator import add
ContextRow2 = namedtuple("context", ["ts","city", "lat", "lon", "moving", "location", "time_day",
                                     "last_app", "last2_app", "app_chain","time_of_day","hour_of_day","weekday","app_continuity","nothing"] )
def addLastApp(row):
    train = row[1][0]
    test = row[1][1]
    if (len(train) == 0 or len(test) == 0):
        #cannot evaluate
        return (row[0], [[],[]])
    temp_list = list(train[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(-1) #last app unknown
    temp_list[1].append(-1) #last2 app unknown
    temp_list[1].append(-1) #app chain unknown
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).day)
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).hour)
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).weekday())
    temp_list[1].append(-1) #app_continuity
    temp_list[1].append(1) #1
    new_row = [[TrainRow(temp_list[0],ContextRow2(*temp_list[1]))]]
    
    ts = test[0].context.ts - train[-1].context.ts
    app_chain_temp = -1 if ts > 60 else train[-1].itemId
    temp_list = list(test[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(train[-1].itemId) #last app 
    temp_list[1].append(-1) #last2 app
    temp_list[1].append(app_chain_temp) #app chain
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).day)
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).hour)
    temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).weekday())
    if ts<=15:
        app_continuity = 2
    elif ts<60:
        app_continuity = 1
    elif ts<120:
        app_continuity = 0
    else:
        app_continuity = -1
    temp_list[1].append(app_continuity) #app_continuity
    temp_list[1].append(1) #1
    new_row.append([TrainRow(temp_list[0],ContextRow2(*temp_list[1]))])
    for i in xrange(1,len(train)):
        ts = train[i].context.ts - train[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else train[i-1].itemId
        temp_list = list(train[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(train[i-1].itemId) #last app 
        if i-2 < 0:
            temp_list[1].append(-1) #last2 app 
        else:
            temp_list[1].append(train[i-2].itemId) #last2 app 
        temp_list[1].append(app_chain_temp) #app chain 
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).day)
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).hour)
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).weekday())
        if ts<=15:
            app_continuity = 2
        elif ts<60:
            app_continuity = 1
        elif ts<120:
            app_continuity = 0
        else:
            app_continuity = -1
        temp_list[1].append(app_continuity)
        temp_list[1].append(1) #1
        new_row[0].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    for i in xrange(1,len(test)):
        ts = test[i].context.ts - test[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else test[i-1].itemId
        temp_list = list(test[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(test[i-1].itemId) #last app 
        if i-2 < 0:
            temp_list[1].append(-1) #last2 app 
        else:
            temp_list[1].append(train[i-2].itemId) #last2 app 
        temp_list[1].append(app_chain_temp) #app chain 
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).day)
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).hour)
        temp_list[1].append(datetime.fromtimestamp(int(temp_list[1][0])).weekday())
        if ts<=15:
            app_continuity = 2
        elif ts<60:
            app_continuity = 1
        elif ts<120:
            app_continuity = 0
        else:
            app_continuity = -1
        temp_list[1].append(app_continuity)  #app_continuity
        temp_list[1].append(1) #1
        new_row[1].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    return (row[0], new_row )
        
splitedRddv2 = splitedRdd.map(addLastApp)
splitedRddv2.take(1)

[(18435,
  [[train(itemId=94874, context=context(ts=1420959850, city=2995, lat=42.897598, lon=-87.853897, moving=1, location=1, time_day=1, last_app=-1, last2_app=-1, app_chain=-1, time_of_day=11, hour_of_day=8, weekday=6, app_continuity=-1, nothing=1)),
    train(itemId=14861, context=context(ts=1420982193, city=2995, lat=42.897644, lon=-87.853874, moving=1, location=2, time_day=2, last_app=94874, last2_app=-1, app_chain=-1, time_of_day=11, hour_of_day=14, weekday=6, app_continuity=-1, nothing=1)),
    train(itemId=94874, context=context(ts=1420982271, city=2995, lat=42.897644, lon=-87.853874, moving=2, location=2, time_day=2, last_app=14861, last2_app=94874, app_chain=-1, time_of_day=11, hour_of_day=14, weekday=6, app_continuity=0, nothing=1)),
    train(itemId=94874, context=context(ts=1420982504, city=2995, lat=42.897675, lon=-87.853874, moving=2, location=2, time_day=2, last_app=94874, last2_app=14861, app_chain=-1, time_of_day=11, hour_of_day=14, weekday=6, app_continuity=-1, not

In [13]:
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores

def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores


def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores  

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()
    
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation2.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[3]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.close()
def print_result3(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation4.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [9]:
def bayesian_context_day(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][10]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][10]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = splitedRddv2.map(bayesian_context_day)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_day')

In [10]:
def bayesian_context_hour(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][11]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][11]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddv2.map(bayesian_context_hour)
print_result2(finalScore, 'bayesian_context_hour')

In [16]:
def bayesian_context_weekday(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][12]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][12]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddv2.map(bayesian_context_weekday)
print_result2(finalScore, 'bayesian_context_weekday')

In [31]:
def bayesian_context_app_continuity(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][13]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][13]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddv2.map(bayesian_context_app_continuity)
print_result2(finalScore, 'bayesian_context_app_continuity')

In [29]:
def bayesian_context_frequency(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][14]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][14]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddv2.map(bayesian_context_frequency)
print_result2(finalScore, 'bayesian_context_frequency')

In [38]:
def countTrain(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    numTrain = len(trainList)
    return line[0],line[1],numTrain
splitedRDDFilter = splitedRddv2.map(countTrain).filter(lambda x:x[2]>600).map(lambda x:(x[0],x[1]))
splitedRDDFilter.take(1)

[(155653,
  [[train(itemId=96547, context=context(ts=1420983906, city=9186, lat=29.614216, lon=-98.463783, moving=1, location=2, time_day=2, last_app=-1, last2_app=-1, app_chain=-1, time_of_day=11, hour_of_day=14, weekday=6, app_continuity=-1, nothing=1)),
    train(itemId=29345, context=context(ts=1420986739, city=9186, lat=29.614296, lon=-98.463501, moving=1, location=2, time_day=2, last_app=96547, last2_app=-1, app_chain=-1, time_of_day=11, hour_of_day=15, weekday=6, app_continuity=-1, nothing=1)),
    train(itemId=60075, context=context(ts=1420986749, city=9186, lat=29.614296, lon=-98.463501, moving=2, location=2, time_day=2, last_app=29345, last2_app=96547, app_chain=29345, time_of_day=11, hour_of_day=15, weekday=6, app_continuity=2, nothing=1)),
    train(itemId=79463, context=context(ts=1420987105, city=9186, lat=29.614296, lon=-98.463501, moving=1, location=2, time_day=2, last_app=60075, last2_app=29345, app_chain=-1, time_of_day=11, hour_of_day=15, weekday=6, app_continuity=-1

In [ ]:
def bayesian_context_app_continuity_big_set(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][13]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][13]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRDDFilter.map(bayesian_context_app_continuity_big_set)
print_result2(finalScore, 'bayesian_context_app_continuity_big_set')

In [ ]:
def bayesian_context_app_continuity_time_window(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][13],t[1][0]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][13],t[1][0]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)
    testList = [x for x in trainList if trainList[numTrain-1][1][0] - x[1][0]<=43200]
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        while (len(trainList)>0 and testList[0][2]-trainList[0][2]>604800):
            trainList.pop(0)
        numTrain = len(trainList)
        context = [(x[0],x[1]) for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        #trainList.pop(0)
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddv2.map(bayesian_context_app_continuity_time_window)
print_result2(finalScore, 'bayesian_context_app_continuity_time_window')

In [55]:
def numberOfApps(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    numApps = len(remove_duplicates([x[0] for x in trainList]))
    
    #numTrain = len(trainList)
    return line[0],line[1],numApps
splitedRDDFilter2 = splitedRddv2.map(numberOfApps).filter(lambda x:x[2]<20).map(lambda x:(x[0],x[1]))
#splitedRDDFilter2 = splitedRddv2.map(numberOfApps)
splitedRDDFilter2.count()

2286

In [65]:
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation4.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [ ]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRDDFilter2.map(mfuFunction)
print_result2(finalScore,'mfu')

In [73]:
def bayesian_context_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRddv2.map(bayesian_context_moving)
print_result2(finalScore, 'bayesian_context_moving')

KeyboardInterrupt: 

In [22]:
def numberOfApps(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    numApps = len(remove_duplicates([x[0] for x in trainList]))
    
    #numTrain = len(trainList)
    return line[0],line[1],numApps
splitedRDDFilter3 = splitedRdd.map(numberOfApps).filter(lambda x:x[2]>50 and x[2]<=60).map(lambda x:(x[0],x[1]))
splitedRDDFilter4 = splitedRdd.map(numberOfApps).filter(lambda x:x[2]>60).map(lambda x:(x[0],x[1]))
#splitedRDDFilter2 = splitedRddv2.map(numberOfApps)
splitedRDDFilter3.count(),splitedRDDFilter4.count()

(10876, 5460)

In [ ]:
from collections import Counter
from operator import add
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRDDFilter3.map(mfuFunction)
print_result3(finalScore,'mfu3')
finalScore = splitedRDDFilter4.map(mfuFunction)
print_result3(finalScore,'mfu4')

In [ ]:
def bayesian_context_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRDDFilter3.map(bayesian_context_moving)
print_result3(finalScore, 'bayesian_context_moving3')
finalScore = splitedRDDFilter4.map(bayesian_context_moving)
print_result3(finalScore, 'bayesian_context_moving4')